In [1]:
import tensorflow as tf

In [2]:
tf.executing_eagerly()

True

In [3]:
x = [[2,]]

In [4]:
m = tf.matmul(x,x)

In [6]:
print(m)

tf.Tensor([[4]], shape=(1, 1), dtype=int32)


In [7]:
m.numpy()

array([[4]], dtype=int32)

In [8]:
a = tf.constant([[1,2],[3,4]])

In [9]:
a

<tf.Tensor: id=3, shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>

In [10]:
a.numpy()

array([[1, 2],
       [3, 4]], dtype=int32)

In [11]:
b  = tf.add(a,1)

In [12]:
b

<tf.Tensor: id=5, shape=(2, 2), dtype=int32, numpy=
array([[2, 3],
       [4, 5]], dtype=int32)>

In [13]:
num  = tf.convert_to_tensor(10)

In [14]:
num

<tf.Tensor: id=6, shape=(), dtype=int32, numpy=10>

In [15]:
for i in range(num.numpy()):
    i =tf.constant(i)
    if int(i %2 )==0:
        print("even")
    else:
        print("odd")

even
odd
even
odd
even
odd
even
odd
even
odd


In [16]:
import numpy as np 

In [18]:
d = np.array([[5,6],[7,8]])

In [21]:
(a+b).numpy()

array([[3, 5],
       [7, 9]], dtype=int32)

In [23]:
v=  tf.Variable(0.0)

In [24]:
v+1

<tf.Tensor: id=50, shape=(), dtype=float32, numpy=1.0>

# 自动求解微分

In [34]:
w = tf.Variable([[1.0]])

In [35]:
with tf.GradientTape() as t:
    loss = w* w

In [36]:
grad = t.gradient(loss,w)

In [37]:
grad

<tf.Tensor: id=86, shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [40]:
w = tf.constant([[1.0]])
with tf.GradientTape() as t:
    t.watch(w)
    loss = w* w
grad = t.gradient(loss,w)

In [41]:
grad

<tf.Tensor: id=96, shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [48]:
w = tf.constant([[1.0]])
with tf.GradientTape(persistent=True) as t:
    t.watch(w)
    y = w* w
    z = y*y
dz_dw = t.gradient(z,w)


In [49]:
dz_dy = t.gradient(z,y)

# mnist数据集实战！

In [121]:
(train_image,train_labels),(test_image,test_labels) = tf.keras.datasets.mnist.load_data()

In [122]:
train_image.shape

(60000, 28, 28)

In [123]:
train_labels.shape

(60000,)

In [124]:
train_image = tf.expand_dims(train_image,-1)
test_image = tf.expand_dims(test_image,-1)

In [125]:
train_image = tf.cast(train_image/255,tf.float32)
test_image = tf.cast(test_image/255,tf.float32)

In [126]:
train_labels = tf.cast(train_labels,tf.int64)
test_labels = tf.cast(test_labels,tf.int64)

In [129]:
dataset = tf.data.Dataset.from_tensor_slices((train_image,train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_image,test_labels))

In [130]:
test_dataset = test_dataset.batch(32)

In [131]:
dataset = dataset.shuffle(60000).repeat().batch(32)

In [103]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),input_shape=(None,None,1)
                           ,activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10),
    
])

In [104]:
optimizer = tf.keras.optimizers.Adam()

In [105]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()

In [106]:
features, label = next(iter(dataset))

In [107]:
prediction = model(features)

In [108]:
prediction.shape

TensorShape([32, 10])

In [109]:
tf.argmax(prediction,axis=-1)

<tf.Tensor: id=320188, shape=(32,), dtype=int64, numpy=
array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2])>

In [110]:
label

<tf.Tensor: id=320175, shape=(32,), dtype=int64, numpy=
array([7, 3, 1, 6, 5, 5, 4, 2, 2, 0, 1, 4, 2, 9, 8, 8, 7, 4, 3, 9, 9, 2,
       5, 9, 2, 5, 1, 5, 0, 3, 0, 1])>

In [111]:
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y,y_)

In [112]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
test_loss = tf.keras.metrics.Mean('train_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [113]:
def train_step(model,images,labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels,pred)
    grads = t.gradient(loss_step,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables ))
    train_loss(loss_step)
    train_accuracy(labels,pred)

In [132]:
def test_step(model,images,labels): #z只有前向传播
    pred = model(images)
    loss_step = loss_func(labels,pred)
    test_loss(loss_step)
    test_accuracy(labels,pred)

In [134]:
def train():
    for epoch in range(10): #epcohs
        for (batch,(images,labels)) in enumerate(dataset):
            train_step(model,images,labels)
        print('epoch{} loss is{} accuracy is{}'
              %(epoch,train_loss.result()
                ,train_accuracy.result()))
        for (batch,(images,labels)) in enumerate(test_dataset):
            test_step(model,images,labels)
        print('epoch{} test_loss is{} test_accuracy is{}'
              %(epoch,train_loss.result()
                ,train_accuracy.result()))
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [135]:
# train()

# tf.keras.metrics汇总计算模块

In [91]:
m = tf.keras.metrics.Mean('acc')

In [92]:
m(10)

<tf.Tensor: id=320021, shape=(), dtype=float32, numpy=10.0>

In [93]:
m(20)

<tf.Tensor: id=320030, shape=(), dtype=float32, numpy=15.0>

In [94]:
m.result()

<tf.Tensor: id=320033, shape=(), dtype=float32, numpy=15.0>

In [95]:
a =tf.keras.metrics.SparseCategoricalAccuracy('acc')

In [97]:
a(label,model(features))

<tf.Tensor: id=320076, shape=(), dtype=float32, numpy=0.0625>

In [98]:
label

<tf.Tensor: id=279, shape=(32,), dtype=int64, numpy=
array([2, 6, 8, 9, 8, 7, 4, 3, 3, 0, 0, 7, 9, 3, 0, 3, 1, 3, 3, 0, 7, 3,
       5, 3, 0, 1, 6, 0, 4, 4, 0, 5])>